In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt

In [2]:
df = pd.read_pickle(r'C:\Users\danys\Downloads\data_compressed.pkl')

In [3]:
df.shape

(5531451, 190)

In [4]:
df_columns = df.columns.to_list()
df_columns[-1]

'D_145'

In [5]:
df.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,...,NaN,NaN,NaN,0.002427,0.003706,0.003818,NaN,0.000569,0.000610,0.002674
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,...,NaN,NaN,NaN,0.003954,0.003167,0.005032,NaN,0.009576,0.005492,0.009217
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954180,0.091505,0.021655,1.009672,0.006815,0.123977,0.007598,0.009423,...,NaN,NaN,NaN,0.003269,0.007329,0.000427,NaN,0.003429,0.006986,0.002603
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960384,0.002455,0.013683,1.002700,0.001373,0.117169,0.000685,0.005531,...,NaN,NaN,NaN,0.006117,0.004516,0.003200,NaN,0.008419,0.006527,0.009600
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947248,0.002483,0.015193,1.000727,0.007605,0.117325,0.004653,0.009312,...,NaN,NaN,NaN,0.003671,0.004946,0.008889,NaN,0.001670,0.008126,0.009827


In [6]:
cat_features = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
num_features = [col for col in df if col not in cat_features + ['S_2', 'customer_ID', 'B_31', 'D_87']]
binary_features = ['B_31', 'D_87']

In [7]:
sampled_df = df.iloc[:15000, :]

In [8]:
from sklearn.impute import SimpleImputer

In [11]:
aggregated_num = sampled_df.groupby('customer_ID')[num_features].agg(['mean', 'median', 'max', 'min', 'std'])

In [13]:
aggregated_cat = sampled_df.groupby('customer_ID')[cat_features].agg(['count', 'nunique'])

In [14]:
df_features = pd.concat([aggregated_cat, aggregated_num], axis=1)
df_features.reset_index(inplace=True)
df_features_sharp = df_features.drop('customer_ID', axis=1)
print(df_features_sharp.shape)

(1251, 897)


C:\Users\danys\AppData\Local\Temp/ipykernel_2844/4085544773.py:3: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_features_sharp = df_features.drop('customer_ID', axis=1)


In [15]:
df_features_sharp.head()

B_30          B_38         D_114         D_116         D_117          ...  \
  count nunique count nunique count nunique count nunique count nunique  ...   
0    13       1    13       1    13       1    13       1    13       1  ...   
1    13       1    13       1    13       1    13       1    13       1  ...   
2    13       1    13       1    13       2    13       1    13       1  ...   
3    13       1    13       1    13       1    13       1    13       2  ...   
4    13       1    13       2    13       1    13       1    13       1  ...   

      D_144                                             D_145            \
       mean    median       max       min       std      mean    median   
0  0.005283  0.005492  0.009616  0.000610  0.002598  0.005814  0.006362   
1  0.004218  0.003169  0.009568  0.000027  0.002871  0.004902  0.004469   
2  0.005113  0.004747  0.009415  0.000129  0.003638  0.004500  0.005110   
3  0.004768  0.004793  0.009919  0.000492  0.002654  0.005236  0.006356   
4  0.004380  0.004070  0.009436  0.000633  0.002633  0.004219  0.002884   

                                 
        max       min       std  
0  0.009827  0.000995  0.003294  
1  0.009390  0.000796  0.002729  
2  0.006932  0.000443  0.002152  
3  0.009836  0.000029  0.003406  
4  0.009666  0.000083  0.003426  

[5 rows x 897 columns]

In [16]:
train_labels = pd.read_csv(r"C:\Users\danys\Downloads\train_new\train_labels.csv")

In [17]:
train_labels_sharp = train_labels.iloc[:1251]

In [18]:
X = df_features_sharp
y = train_labels_sharp.target

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, classification_report
from sklearn.linear_model import SGDClassifier

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= .30, random_state=42)
X_train, X_test = X_train.reset_index(), X_test.reset_index()  # that's done for the sake of starting from 0
X_train, X_test = X_train.drop('index', axis=1), X_test.drop('index', axis=1)

C:\Users\danys\AppData\Local\Temp/ipykernel_2844/2102111229.py:3: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  X_train, X_test = X_train.drop('index', axis=1), X_test.drop('index', axis=1)


In [21]:
model = RandomForestClassifier(
    class_weight={0: 0.20,1: 1.},
    oob_score=True
)

In [22]:
X = df_features_sharp
y = train_labels_sharp.target

In [23]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(X_train)

In [24]:
X_train_imp = imp.transform(X_train)
X_test_imp = imp.transform(X_test)

In [25]:
model.fit(X_train_imp, y_train)
y_pred = model.predict(X_test_imp)

In [26]:
y_pred_proba = model.predict_proba(X_test_imp)[:, model.classes_[model.classes_ == 1][0]]

In [27]:
scores_rf = pd.DataFrame(
        {"scores" : [model.oob_score_, model.score(X_test_imp, y_test), roc_auc_score(y_test, y_pred_proba), f1_score(y_test, y_pred)]},
        index = ["oob_score", "accuracy", "roc_auc", "f1_score"]
)

In [28]:
scores_rf

,scores
oob_score,0.865143
accuracy,0.864362
roc_auc,0.935883
f1_score,0.715084


In [29]:
target_names = ['class 0', 'class 1']

In [30]:
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.90      0.92      0.91       283
     class 1       0.74      0.69      0.72        93

    accuracy                           0.86       376
   macro avg       0.82      0.81      0.81       376
weighted avg       0.86      0.86      0.86       376



In [31]:
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [46]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import KFold, cross_val_score

In [47]:
model_log_regression = LogisticRegressionCV(random_state=42,
                       penalty = 'l1', class_weight= {0: 0.2, 1:1}, cv=5,
                                            solver='saga')

In [48]:
model_log_regression.fit(X_train_imp, y_train)

C:\Users\danys\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\danys\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\danys\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\danys\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\danys\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which mean

C:\Users\danys\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\danys\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\danys\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


LogisticRegressionCV(class_weight={0: 0.2, 1: 1}, cv=5, penalty='l1',
                     random_state=42, solver='saga')

In [49]:
preds_log = model_log_regression.predict(X_test_imp)

In [50]:
predictions_proba_log = model_log_regression.predict_proba(X_test_imp)[:, model_log_regression.classes_[model_log_regression.classes_ == 1][0]]

In [51]:
score_log = pd.DataFrame(
        {roc_auc_score(y_test, predictions_proba_log), f1_score(y_test, preds_log)},
        index = ["roc_auc", "f1-score"], columns=['result']
)

In [52]:
score_log

,result
roc_auc,0.928721
f1-score,0.710744


In [53]:
print(classification_report(y_test, preds_log, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.97      0.78      0.86       283
     class 1       0.58      0.92      0.71        93

    accuracy                           0.81       376
   macro avg       0.77      0.85      0.79       376
weighted avg       0.87      0.81      0.83       376



In [56]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [57]:
from catboost import CatBoostClassifier

In [58]:
cat = CatBoostClassifier(
    iterations=3000,
    learning_rate=0.02,
    depth=3,
)

In [59]:
cat.fit(
    X=X_train,
    y=y_train,
    plot="True",
    early_stopping_rounds=30, 
    eval_set=(X_test, y_test)
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6742055	test: 0.6735661	best: 0.6735661 (0)	total: 131ms	remaining: 6m 32s
1:	learn: 0.6588808	test: 0.6588550	best: 0.6588550 (1)	total: 157ms	remaining: 3m 55s
2:	learn: 0.6464841	test: 0.6462668	best: 0.6462668 (2)	total: 183ms	remaining: 3m 2s
3:	learn: 0.6312492	test: 0.6303104	best: 0.6303104 (3)	total: 208ms	remaining: 2m 35s
4:	learn: 0.6213965	test: 0.6211578	best: 0.6211578 (4)	total: 234ms	remaining: 2m 19s
5:	learn: 0.6070953	test: 0.6065418	best: 0.6065418 (5)	total: 255ms	remaining: 2m 7s
6:	learn: 0.5961852	test: 0.5950321	best: 0.5950321 (6)	total: 276ms	remaining: 1m 57s
7:	learn: 0.5817437	test: 0.5811099	best: 0.5811099 (7)	total: 294ms	remaining: 1m 49s
8:	learn: 0.5706754	test: 0.5691505	best: 0.5691505 (8)	total: 311ms	remaining: 1m 43s
9:	learn: 0.5606787	test: 0.5603178	best: 0.5603178 (9)	total: 329ms	remaining: 1m 38s
10:	learn: 0.5497129	test: 0.5484309	best: 0.5484309 (10)	total: 345ms	remaining: 1m 33s
11:	learn: 0.5424327	test: 0.5410806	best: 

97:	learn: 0.2663322	test: 0.2949587	best: 0.2949587 (97)	total: 1.74s	remaining: 51.4s
98:	learn: 0.2652996	test: 0.2941392	best: 0.2941392 (98)	total: 1.75s	remaining: 51.4s
99:	learn: 0.2642030	test: 0.2933445	best: 0.2933445 (99)	total: 1.77s	remaining: 51.2s
100:	learn: 0.2632127	test: 0.2928929	best: 0.2928929 (100)	total: 1.78s	remaining: 51.1s
101:	learn: 0.2624352	test: 0.2924033	best: 0.2924033 (101)	total: 1.79s	remaining: 51s
102:	learn: 0.2613707	test: 0.2919646	best: 0.2919646 (102)	total: 1.81s	remaining: 50.9s
103:	learn: 0.2602762	test: 0.2918726	best: 0.2918726 (103)	total: 1.82s	remaining: 50.8s
104:	learn: 0.2594108	test: 0.2917438	best: 0.2917438 (104)	total: 1.84s	remaining: 50.7s
105:	learn: 0.2586417	test: 0.2912813	best: 0.2912813 (105)	total: 1.85s	remaining: 50.6s
106:	learn: 0.2572897	test: 0.2902616	best: 0.2902616 (106)	total: 1.87s	remaining: 50.5s
107:	learn: 0.2563807	test: 0.2898746	best: 0.2898746 (107)	total: 1.88s	remaining: 50.4s
108:	learn: 0.2552

201:	learn: 0.2023658	test: 0.2734241	best: 0.2734241 (201)	total: 3.36s	remaining: 46.5s
202:	learn: 0.2020805	test: 0.2734558	best: 0.2734241 (201)	total: 3.38s	remaining: 46.5s
203:	learn: 0.2018402	test: 0.2733479	best: 0.2733479 (203)	total: 3.39s	remaining: 46.5s
204:	learn: 0.2012407	test: 0.2734227	best: 0.2733479 (203)	total: 3.41s	remaining: 46.5s
205:	learn: 0.2008413	test: 0.2732527	best: 0.2732527 (205)	total: 3.43s	remaining: 46.5s
206:	learn: 0.2004861	test: 0.2731682	best: 0.2731682 (206)	total: 3.44s	remaining: 46.4s
207:	learn: 0.2000263	test: 0.2731535	best: 0.2731535 (207)	total: 3.46s	remaining: 46.4s
208:	learn: 0.1995633	test: 0.2731170	best: 0.2731170 (208)	total: 3.48s	remaining: 46.4s
209:	learn: 0.1991298	test: 0.2730658	best: 0.2730658 (209)	total: 3.49s	remaining: 46.4s
210:	learn: 0.1987258	test: 0.2730263	best: 0.2730263 (210)	total: 3.51s	remaining: 46.4s
211:	learn: 0.1983606	test: 0.2729673	best: 0.2729673 (211)	total: 3.52s	remaining: 46.3s
212:	learn

In [60]:
y_pred_cat = cat.predict(X_test)
y_pred_proba_cat = cat.predict_proba(X_test)[:,cat.classes_[cat.classes_ == 1][0]]
scores = pd.DataFrame(
        {"scores" : [ cat.score(X_test, y_test), roc_auc_score(y_test, y_pred_proba), f1_score(y_test, y_pred)]},
        index = [ "accuracy", "roc_auc", "f1_score"]
)
scores

,scores
accuracy,0.882979
roc_auc,0.935883
f1_score,0.715084
